<a href="https://colab.research.google.com/github/hilaler/Sentiment-Analysis-on-Indonesia-Tweet-Dataset/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%tensorflow_version 2.x

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
data = pd.read_csv("indo_dataset.csv")

In [7]:
data.iloc[0]

index                                                       0
sentimen                                                    0
Tweet       lagu bosan apa yang aku save ni huhuhuhuhuhuhu...
Name: 0, dtype: object

In [8]:
max_fatures = 2500
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Tweet'].values)

In [9]:
X = tokenizer.texts_to_sequences(data['Tweet'].values)

In [10]:
X[0]

[629, 983, 15, 3, 1, 1735, 17]

In [11]:
X = pad_sequences(X)

In [12]:
X[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          4,  103,  244,   11,  668,   20,    5,    1, 1736,   25,   53,
       2068,   61,   49], dtype=int32)

In [13]:
Y = pd.get_dummies(data['sentimen']).values

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)

In [15]:
X.shape[1]

25

In [16]:
embed_dim = 128

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 128)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 459,970
Trainable params: 459,970
Non-trainable params: 0
__________________________________________________

In [17]:
batch_size = 32
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 10, batch_size=batch_size)

Epoch 1/10
115/115 [==============================] - 9s 52ms/step - loss: 0.6679 - accuracy: 0.5738 - val_loss: 0.6035 - val_accuracy: 0.6893
Epoch 2/10
115/115 [==============================] - 5s 47ms/step - loss: 0.4939 - accuracy: 0.7670 - val_loss: 0.5686 - val_accuracy: 0.7186
Epoch 3/10
115/115 [==============================] - 5s 47ms/step - loss: 0.3414 - accuracy: 0.8575 - val_loss: 0.6812 - val_accuracy: 0.7037
Epoch 4/10
115/115 [==============================] - 5s 47ms/step - loss: 0.2596 - accuracy: 0.8986 - val_loss: 0.8311 - val_accuracy: 0.7120
Epoch 5/10
115/115 [==============================] - 5s 45ms/step - loss: 0.1937 - accuracy: 0.9251 - val_loss: 1.1949 - val_accuracy: 0.7175
Epoch 6/10
115/115 [==============================] - 5s 48ms/step - loss: 0.1328 - accuracy: 0.9477 - val_loss: 1.3652 - val_accuracy: 0.7076
Epoch 7/10
115/115 [==============================] - 6s 50ms/step - loss: 0.1040 - accuracy: 0.9567 - val_loss: 1.5924 - val_accuracy: 0.7120

In [18]:
X_T = tokenizer.texts_to_sequences(np.array(["saya benci baso", "saya suka baso"]))
X_T = pad_sequences(X_T, maxlen=25)

In [19]:
for d in np.argmax(model.predict(X_T), axis=0):
  print(["Negativ", "Positif"][d])

Negativ
Positif


In [20]:
model.save_weights('lstm.h5')

In [21]:
model2 = Sequential()
model2.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model2.add(LSTM(256))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(2, activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 128)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 730,818
Trainable params: 730,818
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
model2.load_weights('lstm.h5')

In [24]:
for d in np.argmax(model2.predict(X_T), axis=0):
  print(["Negativ", "Positif"][d])

Positif
Negativ
